In [39]:
# Importing for system files
import os
import sys
import csv
import pprint

In [40]:
# Loading environmental variables 
from dotenv import load_dotenv
load_dotenv()

True

In [41]:
# Import System Files
dir_path = os.path.dirname(os.path.realpath('__file__'))
sys.path.append(dir_path + '/lib/')  
from process_work import process_work

In [42]:
# For processing WhatsApp data file
from whatstk import df_from_txt_whatsapp
from whatstk.data import whatsapp_urls


In [43]:
# Date before which chat is not scored
# cut_off_date = 20240607
# file_key = '20240912_beerhall_chat'

cut_off_date = 20240605
file_key = '20240912_townhall_chat'

# file_key = '20231213_townhall_chat'
# file_key = '20231213_beerhall_chat'
# file_key = '20231214_agroverse_chat'
# file_key = '20231214_main_dao_website_chat'
# file_key = '20240603_agroverse_chat'
# file_key = '20240608_sunmint'



In [44]:
# Importing WhatsApp data file
df = df_from_txt_whatsapp(filepath=f"./data/{file_key}.txt")
df.head(100)

# Setup work analysis file
analysis_file = open(f"analysis/{file_key}.csv", 'w')
analysis_writer = csv.writer(analysis_file)

header = ['Contributor Name', 'Project Name', 'Contribution Made', 'Rubric classification', 'TDGs Provisioned', 'Status', 'TDGs Issued', 'Status date']
# data = ['Afghanistan', 652090, 'AF', 'AFG']
analysis_writer.writerow(header)

pp = pprint.PrettyPrinter(indent=4)

In [45]:
# For OpenAI client
from openai import OpenAI

# Acquire OpenAI API Key via this URL 
#    https://platform.openai.com/api-keys
# Save Secret in .env file with 
#    OPENAI_API_KEY=YOUR SECRET KEY

openai_client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [46]:
total_rows = df.count()['date']

In [47]:
# Calling OpenAi API to get time taken for work done
def check_tdg_issued(work_description):
    payload = f"We award 100TDG for every hour of contribution made that was explicitly reported in the Action Description. \
                We award 1.65TDG for every minute of contribution made that was explicitly reported in the Action Description. \
                 Amount of TDG issued should be prorated based on actual time explicitly reported\
                 Time taken indicated in the 'Action Description' may also appear in the following formats as a stand alone sentences\
                     Time Taken : X Hours \
                     Time Taken : Y Minutes \
                     \
                 We award 1TDG for each 1USD of expenses paid for on behalf by DAO member \
                 There are only two available options for 'Rubric classification' and they are: \
                     '100TDG For every 1 hour of human effort' \
                     '1TDG For every 1 USD of liquidity injected' \
                     \
                 Response must be short and concise\
                 Response must be in a single line CSV format. Response must not include headers\
                     Rubric classification (String); TDGs Issued (2 decimal float)\
                     \
                 If Action Description does not make sense return the following values \
                     Unknown; 0 \
                     \
                 For the following 'Action Description' determine Rubric classification and TDGs Issued \
                     \
                 Action Description:\
                 {work_description}."
    
    work_process_completion = openai_client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": payload,
            }
        ],
        model="gpt-3.5-turbo",
    )
    return work_process_completion.choices[0].message.content

# response = check_tdg_issued("Gary Teh walked the dog for 1 hour")

In [48]:
format = '%Y%m%d'
for index, row in df.iterrows():
    # Doesn't process record if occurred before a specific date
    if int(row['date'].strftime(format)) < cut_off_date:
        continue
    
    clean_work_description = row['message'].encode('ascii','ignore') 
    response = check_tdg_issued(clean_work_description)
    openai_result = response.split(';')
    
    classification = "Unknown"
    tdg_issue = 0
    the_status = "Successfully Completed / Full Provision Awarded"
    
    if len(openai_result) == 2:
        classification = openai_result[0]
        tdg_issue = openai_result[1]
        
    if classification == "Unknown":
        the_status = ""

    # header = ['Contributor Name', 'Project Name', 'Contribution Made', 'Rubric classification', 'TDGs Provisioned', 'Status', 'TDGs Issued', 'Status date']
    data = [
        row['username'], 
        '',
        row['message'], 
        classification, 
        tdg_issue, 
        the_status,
        tdg_issue, 
        row['date'].strftime(format)
    ]
    print("")
    print(f"Processing {index} of {total_rows}: ")
    pp.pprint(response)
    pp.pprint(openai_result)
    pp.pprint(len(openai_result))
    pp.pprint(data)
    
    
    analysis_writer.writerow(data)        


Processing 1672 of 1750: 
'100TDG For every 1 hour of human effort; 15.00'
['100TDG For every 1 hour of human effort', ' 15.00']
2
[   'Gary Teh',
    '',
    'Spent past 30 minutes tabulating chat logs within this channel for TDG '
    'issues on our ledger.\n'
    '\n'
    'https://docs.google.com/spreadsheets/d/1I9V6mm_WQm4ArQ60MBlP2AEUFdlTd7Ys80ua9e4zyUQ/edit#gid=1984232215\n'
    '\n'
    'Am done till line 136. \n'
    '\n'
    '_PS: heading over to meet @14152032783 for venue setup for summit '
    'tomorrow. Will continue this process tomorrow._\n'
    '\n'
    '_Our ledger which includes tabulated contributions till line 136 on raw '
    'chat log_\n'
    '_https://truesight.me/ledger_ \u200e<This message was edited>',
    '100TDG For every 1 hour of human effort',
    ' 15.00',
    'Successfully Completed / Full Provision Awarded',
    ' 15.00',
    '20240606']

Processing 1673 of 1750: 
'Unknown; 0'
['Unknown', ' 0']
2
[   '<ops> The Town Hall',
    '',
    '\u200e\u200eDis

In [49]:
analysis_file.close()

In [50]:
# print(f"We award 100TDG for every hour of contribution made that was explicitly reported in the Action Description. \
#                 We award 1.666666666666667TDG for every minute of contribution made that was explicitly reported in the Action Description. \
#                  Amount of TDG issued should be prorated based on actual time explicitly reported\
#                  Time taken indicated in the 'Action Description' may also appear in the following formats as a stand alone sentences\
#                      Time Taken : X Hours \
#                      Time Taken : Y Minutes \
#                      \
#                  We award 1TDG for every 1USD of expenses paid for onbehalf by DAO member \
#                  There are only two available options for 'Rubric classification' and they are: \
#                      'Contributed Fiat Funding' \
#                      'Contributed Human Effort' \
#                      \
#                  Response must be short and concise\
#                  Response must be in a single line CSV format. Response must not include headers\
#                      Rubric classification (String); TDGs Issued (Integer)\
#                      \
#                  If Action Description does not make sense return the following values \
#                      Unknown; 0 \
#                      \
#                  For the following 'Action Description' determine Rubric classification and TDGs Issued \
#                      \
#                  Action Description:\
#                  Text submission by DAO member in WhatsApp channels.")


We award 100TDG for every hour of contribution made that was explicitly reported in the Action Description.                 We award 1.666666666666667TDG for every minute of contribution made that was explicitly reported in the Action Description.                  Amount of TDG issued should be prorated based on actual time explicitly reported                 Time taken indicated in the 'Action Description' may also appear in the following formats as a stand alone sentences                     Time Taken : X Hours                      Time Taken : Y Minutes                                       We award 1TDG for every 1USD of expenses paid for onbehalf by DAO member                  There are only two available options for 'Rubric classification' and they are:                      'Contributed Fiat Funding'                      'Contributed Human Effort'                                       Response must be short and concise                 Response must be in a single line CSV format